In [1]:
from utils_nlp.models.pretrained_embeddings.fasttext import load_pretrained_vectors
from utils_nlp.dataset.muse import load_pandas_df
from utils_nlp.models.geomm.utils import LangEdge, LangMapping, evaluate_lang_pair, create_train_mapping, create_dictionary
from utils_nlp.models.geomm.geomm import get_geomm_mapping
import numpy as np
import collections
import tensorflow as tf

unable to import 'smart_open.gcs', disabling that module


In [2]:
# Set the path for where your repo is located
NLP_REPO_PATH = os.path.join('..','..')

# Set the path for where your datasets are located
BASE_DATA_PATH = os.path.join(NLP_REPO_PATH, "data")

In [3]:
src_label = 'en'
trg_label = 'es'

#### Note: It's prefereable to use one of the followin instead of using the download option provided in the code as it is currently not able to download the full file.

```sh 
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.<lang_code>.vec -o ./data/embedding/fastText/
curl https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.<lang_code>.vec -o ./data/embedding/fastText/wiki.<lang_code>.vec
```



In [4]:
embeddings = {}
embeddings[src_label] = load_pretrained_vectors(BASE_DATA_PATH + 'embedding', file_name=f"wiki.{src_label}.vec", limit=200000)
embeddings[trg_label] = load_pretrained_vectors(BASE_DATA_PATH + 'embedding', file_name=f"wiki.{trg_label}.vec", limit=200000)

Vector file already exists. No changes made. ../../data/embedding/fastText/wiki.en.vec
Vector file already exists. No changes made. ../../data/embedding/fastText/wiki.es.vec


In [5]:
pairs_data_train = load_pandas_df(local_cache_path=BASE_DATA_PATH + 'muse', file_split='train', src_language=src_label, trg_language=trg_label)

In [6]:
pairs_data_test = load_pandas_df(local_cache_path=BASE_DATA_PATH + 'muse', file_split='test', src_language=src_label, trg_language=trg_label)

In [7]:
train_mapping = create_train_mapping(pairs_data_train, embeddings[src_label], embeddings[trg_label])

In [8]:
edge = LangEdge(src_label, trg_label, mapping=train_mapping)

In [9]:
geomm_result = get_geomm_mapping([edge], embeddings, maxiter=500, lr=1e2, verbosity=3, tf_dtype=tf.float64)





Compiling cost function...
Computing gradient of cost function...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Optimizing...
 iter		   cost val	    grad. norm
    0	+1.4568397289208996e+08	2.11438722e+08
    1	+1.3781458252634510e+08	8.06798000e+07
    2	+1.3435562836160496e+08	7.14910067e+07
    3	+1.2920747839624549e+08	6.33624851e+07
    4	+1.2559508879070008e+08	7.03882761e+07
    5	+1.1836904383738501e+08	4.00585058e+07
    6	+1.1489238662347515e+08	3.65685490e+07
    7	+1.1016326677983080e+08	5.96899608e+07
    8	+9.9339773456328198e+07	3.36556405e+07
    9	+9.3328002088405669e+07	5.23636211e+07
   10	+8.3024200081805795e+07	2.44817316e+07
   11	+7.9763620563963711e+07	1.72261331e+07
   12	+7.6283378233783379e+07	1.41511287e+07
   13	+7.2997515892599687e+07	1.73692447e+07
   14	+6.0912301273600221e+07	2.23295777e+07
   15	+5.6626993415713347e+07	1.30735612e+07
   16	+5.3189107115437202e+07	8.74451118e+06
   17	+5.08002480235431

  159	-2.2024327943897342e+02	4.71078341e+00
  160	-2.2054756378407876e+02	3.22399211e+00
  161	-2.2071167652631814e+02	2.55544161e+00
  162	-2.2083182943829203e+02	2.06058680e+00
  163	-2.2091664176912585e+02	1.88138067e+00
  164	-2.2099648484563266e+02	1.78737977e+00
  165	-2.2107931180175635e+02	1.54823104e+00
  166	-2.2119500410447634e+02	2.34053958e+00
  167	-2.2137481020633129e+02	3.88601864e+00
  168	-2.2155132081643123e+02	3.06356701e+00
  169	-2.2166111701714868e+02	3.07768742e+00
  170	-2.2179800213104164e+02	2.79021547e+00
  171	-2.2195073859734202e+02	2.26884231e+00
  172	-2.2204966273397164e+02	2.33034284e+00
  173	-2.2216950654923090e+02	2.17804240e+00
  174	-2.2234772442005828e+02	3.17491337e+00
  175	-2.2247391019265302e+02	2.17228586e+00
  176	-2.2251016933252663e+02	1.49229780e+00
  177	-2.2253603522672012e+02	1.32262412e+00
  178	-2.2258274975024597e+02	1.71155016e+00
  179	-2.2261954205888219e+02	1.41068457e+00
  180	-2.2264518407558327e+02	1.41983477e+00
  181	-2.2

  345	-2.2491702749135541e+02	1.04091159e-01
  346	-2.2491745147928634e+02	1.59882263e-01
  347	-2.2491797199405056e+02	1.51853236e-01
  348	-2.2491887995308844e+02	2.48686748e-01
  349	-2.2491995698098140e+02	1.71051374e-01
  350	-2.2492050442991979e+02	1.60575702e-01
  351	-2.2492110459430847e+02	1.27252577e-01
  352	-2.2492161156593454e+02	2.09399644e-01
  353	-2.2492265176866286e+02	3.03969489e-01
  354	-2.2492373827977784e+02	2.32879924e-01
  355	-2.2492443292914896e+02	2.62206410e-01
  356	-2.2492622929612440e+02	3.67539975e-01
  357	-2.2492814015722183e+02	3.21428220e-01
  358	-2.2492990319606719e+02	2.26035345e-01
  359	-2.2493069577969385e+02	1.91679013e-01
  360	-2.2493136781473106e+02	1.65471519e-01
  361	-2.2493192342209923e+02	1.41809923e-01
  362	-2.2493232884747795e+02	1.54507866e-01
  363	-2.2493283289042589e+02	1.07561697e-01
  364	-2.2493323952664457e+02	1.40988046e-01
  365	-2.2493362287240228e+02	1.02982865e-01
  366	-2.2493406501814425e+02	1.47731216e-01
  367	-2.2

In [10]:
test_dictionary = create_dictionary(pairs_data_test, embeddings[src_label], embeddings[trg_label])

In [11]:
evaluate_lang_pair(embeddings[src_label].vectors, embeddings[trg_label].vectors, geomm_result[1][src_label], geomm_result[1][trg_label], geomm_result[0], test_dictionary)

100%|██████████| 3/3 [00:30<00:00, 10.07s/it]


{'Cosine Accuracy': 0.7893333333333333, 'CSLS Accuracy': 0.8133333333333334}